##### Imports

In [1]:
import sys, os
from asvFormula.digraph import *
from asvFormula.classesSizes.algorithmTime import *

from importlib import reload 
import asvFormula
import asvFormula.topoSorts.topoSortsCalc as tp
import asvFormula.topoSorts.toposPositions as pos
import asvFormula.testingFunctions as tst


tp = reload(asvFormula.topoSorts.topoSortsCalc)
pos = reload(asvFormula.topoSorts.toposPositions)
tst = reload(asvFormula.testingFunctions)

In [2]:
def pathToLeaf(tree : nx.DiGraph, start_node):
    path = [start_node]
    current = start_node
    while not isLeaf(current, tree):
        current = next(tree.successors(current))
        path.append(current)
    return path

### Positions of node in toposorts


In [3]:
numLevels = 3
branchingFactor = 3

treeTest = balancedTree(numLevels, branchingFactor)

middleTree = pathToLeaf(treeTest, 0)
#allTopos = list(nx.all_topological_sorts(treeTest))
#tst.test_allToposPositions(middleTree, treeTest, allTopos)

# It takes 8 minutes to run. Most of the time is spent calculating all the topological sorts and iterating over them

### Toposorts of a polytree


#### Polytree with one intersection


In [ ]:
numLevels = 2
branchingFactor = 2

leftTree = balancedTree(numLevels, branchingFactor)
rigthTree = balancedTree(numLevels, branchingFactor)
middleTree = balancedTree(numLevels, branchingFactor)

union = nx.union(leftTree, rigthTree, rename=('1-', '2-'))
union = nx.union(union, middleTree, rename=('', '3-'))

#Connect the trees to the middle one
union.add_edge('1-2', '3-0')
union.add_edge('2-1', '3-0')
#drawGraph(union)
tst.test_allPolyTopos(union)



In [ ]:
numLevels = 2
branchingFactor = 2

leftTree = balancedTree(numLevels, 1)
rigthTree = balancedTree(numLevels, 2)
middleTree = balancedTree(1, branchingFactor)

union = nx.union(leftTree, rigthTree, rename=('1', '2'))
union = nx.union(union, middleTree, rename=('', '3'))

#Connect the trees to the middle one
union.add_edge('11', '30')
union.add_edge('21', '30')
drawGraph(union)
tst.test_allPolyTopos(union)



In [31]:
import itertools
# I want to classify the topological sorts depending on the order of some nodes

parentNodes = ['11', '21', '30']

def classifyNodesByOrder(parentNodes):
    allTopos = list(nx.all_topological_sorts(union))
    permutations = list(itertools.permutations(parentNodes))
    toposPermutations = {}

    for topo in allTopos:
        for perm in permutations:
            if [topo.index(node) for node in perm] == sorted([topo.index(node) for node in perm]):
                toposPermutations[perm] = toposPermutations.get(perm, 0) + 1

    return toposPermutations

classifyNodesByOrder(parentNodes)


{('21', '11', '30'): 14, ('11', '21', '30'): 12}